In [84]:
import pandas as pd
import numpy as np
train = pd.read_csv('C:/Users/owner/Desktop/구내식당/train.csv')
test =  pd.read_csv('C:/Users/owner/Desktop/구내식당/test.csv')
submission =  pd.read_csv('C:/Users/owner/Desktop/구내식당/sample_submission.csv')

In [85]:
train= train[train['석식계']!=0] # 석식 이용 인원 0인 날 삭제 (= 이상치 삭제)

train.loc[train['요일']=='월','요일']=1 # 요일을 숫자로 변경
train.loc[train['요일']=='화','요일']=2
train.loc[train['요일']=='수','요일']=3
train.loc[train['요일']=='목','요일']=4
train.loc[train['요일']=='금','요일']=5

def month(text:str): # 월 칼럼 추가
    return int(text[5:7])
train['월'] = train['일자'].map(month)

def day(text:str): # 일 칼럼 추가
    return int(text[-2:])
train['일'] = train['일자'].map(day)


# test= test[test['석식계']!=0] # 석식 이용 인원 0인 날 삭제 (= 이상치 삭제)  => test 셋에는 중식계, 석식계 칼럼이 없기 때문에 삭제 할 필요가 없음

test.loc[test['요일']=='월','요일']=1 # 요일을 숫자로 변경
test.loc[test['요일']=='화','요일']=2
test.loc[test['요일']=='수','요일']=3
test.loc[test['요일']=='목','요일']=4
test.loc[test['요일']=='금','요일']=5

def month(text:str): # 월 칼럼 추가
    return int(text[5:7])
test['월'] = test['일자'].map(month)

def day(text:str): # 일 칼럼 추가
    return int(text[-2:])
test['일'] = test['일자'].map(day)

In [86]:
train['현재원']=train['본사정원수']-train['본사휴가자수']-train['본사출장자수']-train['현본사소속재택근무자수']
test['현재원']=test['본사정원수']-test['본사휴가자수']-test['본사출장자수']-test['현본사소속재택근무자수'] # test셋에도 현재원을 생성해줌!

In [87]:
features = ['요일', '월', '일' , '현재원' ,'본사휴가자수',	'본사출장자수'	,'본사시간외근무명령서승인건수'	,'현본사소속재택근무자수']
train[features]

,요일,월,일,현재원,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수
0,1,2,1,2401.0,50,150,238,0.0
1,2,2,2,2378.0,50,173,319,0.0
2,3,2,3,2365.0,56,180,111,0.0
3,4,2,4,2277.0,104,220,355,0.0
4,5,2,5,2142.0,278,181,34,0.0
...,...,...,...,...,...,...,...,...
1200,3,1,20,2319.0,75,198,4,391.0
1201,4,1,21,2309.0,92,231,462,351.0
1202,5,1,22,2177.0,255,248,1,303.0
1203,1,1,25,2396.0,107,153,616,327.0


In [88]:
test['현재원']=test['본사정원수']-test['본사휴가자수']-test['본사출장자수']-test['현본사소속재택근무자수'] # test셋에도 현재원을 생성해줌!
test[features].head()

,요일,월,일,현재원,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수
0,3,1,27,2355.0,88,182,5,358.0
1,4,1,28,2319.0,104,212,409,348.0
2,5,1,29,2170.0,270,249,0,294.0
3,1,2,1,2340.0,108,154,538,322.0
4,2,2,2,2362.0,62,186,455,314.0


In [89]:
X= train[features]
y= train['중식계']

from sklearn.tree import DecisionTreeRegressor
lunch_model= DecisionTreeRegressor()

lunch_model.fit(X,y)

lunch_count_predict= lunch_model.predict(test[features])

DecisionTreeRegressor()

In [90]:
features.append('중식계') # 석식계를 예측할땐 중식계도 같이 피처로 놓고 예측을 해야하기 때문에 중식계라는 피처를 추가해줬다.
features

['요일',
 '월',
 '일',
 '현재원',
 '본사휴가자수',
 '본사출장자수',
 '본사시간외근무명령서승인건수',
 '현본사소속재택근무자수',
 '중식계']

In [91]:
test['중식계']= lunch_count_predict # 중식계를 예측한 모델을 test셋에 추가해줌

##  DecisionTreeRegressor

In [92]:
X= train[features]
y= train['석식계']

from sklearn.tree import DecisionTreeRegressor
dinner_model= DecisionTreeRegressor()

dinner_model.fit(X,y)

dinner_count_predict= dinner_model.predict(test[features]) 

DecisionTreeRegressor()

In [93]:
# plt.plot(dinner_count_predict)
# plt.show()

# RandomForestRegressor

In [94]:
features=features[:-1] # 중식계 제거

X=train[features]
y=train['중식계']

from sklearn.ensemble import RandomForestRegressor

lunch_model = RandomForestRegressor()
lunch_model.fit(X,y)
lunch_count_predict=lunch_model.predict(test[features])

# plt.plot(lunch_count_predict)
# plt.show()

RandomForestRegressor()

In [95]:
test['중식계']= lunch_count_predict # 중식계 추가

X=train[features]
y=train['석식계']

dinner_model = RandomForestRegressor()
dinner_model.fit(X,y)
dinner_count_predict=dinner_model.predict(test[features])

# plt.plot(dinner_count_predict)
# plt.show()

RandomForestRegressor()

## 하이퍼 파라미터 튜닝

In [96]:
features= features[:-1]

X=train[features]
y=train['중식계']

lunch_model = RandomForestRegressor(n_estimators=200, criterion="mae")
lunch_model.fit(X,y)
lunch_count_predict=lunch_model.predict(test[features])

# plt.plot(lunch_count_predict)
# plt.show()

RandomForestRegressor(criterion='mae', n_estimators=200)

In [97]:
features.append('중식계')

test['중식계']=lunch_count_predict

X=train[features]
y=train['석식계']

dinner_model = RandomForestRegressor(n_estimators=200, criterion="mae")
dinner_model.fit(X,y)
dinner_count_predict=dinner_model.predict(test[features])

# plt.plot(dinner_count_predict)
# plt.show()

RandomForestRegressor(criterion='mae', n_estimators=200)

In [98]:
train["요일"] = pd.to_numeric(train["요일"], errors='coerce')
test["요일"] = pd.to_numeric(test["요일"], errors='coerce')

In [99]:
features = ['요일','월','일','현재원','본사휴가자수',
 '본사출장자수','본사시간외근무명령서승인건수','현본사소속재택근무자수',
 '중식계']


In [100]:
X_train=train[['요일','월','일','현재원','본사휴가자수',
 '본사출장자수','본사시간외근무명령서승인건수','현본사소속재택근무자수']]
y_train=train['중식계']

X_test=test[['요일','월','일','현재원','본사휴가자수',
 '본사출장자수','본사시간외근무명령서승인건수','현본사소속재택근무자수','중식계']]
# y_test=test[['석식계']]

In [101]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1162 entries, 0 to 1204
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1162 non-null   object 
 1   요일              1162 non-null   int64  
 2   본사정원수           1162 non-null   int64  
 3   본사휴가자수          1162 non-null   int64  
 4   본사출장자수          1162 non-null   int64  
 5   본사시간외근무명령서승인건수  1162 non-null   int64  
 6   현본사소속재택근무자수     1162 non-null   float64
 7   조식메뉴            1162 non-null   object 
 8   중식메뉴            1162 non-null   object 
 9   석식메뉴            1162 non-null   object 
 10  중식계             1162 non-null   float64
 11  석식계             1162 non-null   float64
 12  월               1162 non-null   int64  
 13  일               1162 non-null   int64  
 14  현재원             1162 non-null   float64
dtypes: float64(4), int64(7), object(4)
memory usage: 145.2+ KB


# XGBRegressor

In [102]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBRegressor

param = {'max_depth':[2],
    'n_estimators':[100],
    'min_child_weight': [4] }


X_train=train[['요일','월','일','현재원','본사휴가자수',
 '본사출장자수','본사시간외근무명령서승인건수','현본사소속재택근무자수']]
y_train=train['중식계']
X_test= test[['요일','월','일','현재원','본사휴가자수',
 '본사출장자수','본사시간외근무명령서승인건수','현본사소속재택근무자수']]
    
model = xgb.XGBRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param, cv=10, 
                           scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_par

{'max_depth': 2, 'min_child_weight': 4, 'n_estimators': 100}


In [103]:
lunch_count_predict = grid_search.predict(X_test)

In [104]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBRegressor

X_train=train[['요일','월','일','현재원','본사휴가자수',
 '본사출장자수','본사시간외근무명령서승인건수','현본사소속재택근무자수','중식계']]
y_train=train['석식계']
X_test=test[['요일','월','일','현재원','본사휴가자수',
 '본사출장자수','본사시간외근무명령서승인건수','현본사소속재택근무자수','중식계']]

param = {
    'max_depth':[2,3],
    'n_estimators':[100,200],
    'min_child_weight': [4],
}
    
model = xgb.XGBRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param, cv=10, 
                           scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_par

{'max_depth': 2, 'min_child_weight': 4, 'n_estimators': 100}


In [105]:
dinner_count_predict = grid_search.predict(X_test)

In [107]:
submission['중식계']= lunch_count_predict
submission['석식계']= dinner_count_predict
submission

,일자,중식계,석식계
0,2021-01-27,1003.965210,372.503693
1,2021-01-28,892.123596,409.162415
2,2021-01-29,607.488342,226.607788
3,2021-02-01,1242.130615,485.161835
4,2021-02-02,1028.931030,511.574585
5,2021-02-03,929.872620,459.012268
6,2021-02-04,928.802673,514.182007
7,2021-02-05,722.784302,367.772034
8,2021-02-08,1292.840820,652.297729
9,2021-02-09,1083.086060,523.939697


In [110]:
submission.to_csv('submission_XGB제출.csv', index=False)